In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define the image size and batch size
image_size = (256, 256)
batch_size = 32

# Define the path to your data directory in your drive
data_dir = '/gdrive/My Drive/brainTumorClassification'

# Use tf.keras.preprocessing.image_dataset_from_directory() to load the data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=image_size,
  batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=image_size,
  batch_size=batch_size
)

# Define the DARTNet model architecture
def conv_block(inputs, filters, kernel_size, strides, padding):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def dense_block(inputs, num_layers, growth_rate):
    x = inputs
    for i in range(num_layers):
        conv = conv_block(x, growth_rate, (3,3), strides=1, padding='same')
        x = layers.Concatenate()([x, conv])
    return x

def transition_block(inputs, compression_factor):
    filters = int(inputs.shape[-1] * compression_factor)
    x = conv_block(inputs, filters, (1,1), strides=1, padding='same')
    x = layers.AveragePooling2D((2,2), strides=2)(x)
    return x

inputs = layers.Input(shape=(256, 256, 3))

x = conv_block(inputs, 64, (7,7), strides=2, padding='same')
x = layers.MaxPooling2D((3,3), strides=2, padding='same')(x)

x = dense_block(x, num_layers=6, growth_rate=32)
x = transition_block(x, compression_factor=0.5)

x = dense_block(x, num_layers=12, growth_rate=32)
x = transition_block(x, compression_factor=0.5)

x = dense_block(x, num_layers=48, growth_rate=32)
x = transition_block(x, compression_factor=0.5)

x = dense_block(x, num_layers=32, growth_rate=32)
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Fit the model to the data
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(val_ds, verbose=2)
print('Test accuracy:', test_acc)

Found 3264 files belonging to 2 classes.
Using 2612 files for training.
Found 3264 files belonging to 2 classes.
Using 652 files for validation.
Epoch 1/10
82/82 [==============================] - 4932s 59s/step - loss: 0.4258 - accuracy: 0.8587 - val_loss: 34.0237 - val_accuracy: 0.9034
Epoch 2/10
 7/82 [=>............................] - ETA: 1:07:30 - loss: 0.3576 - accuracy: 0.8438